In [ ]:
from youtube_agent_utils import (
    OPEN_AI_API_KEY,
    get_authenticated_youtube_api,
    get_views_snippet,
    update_video_title,
    get_last_n_videos_with_views,
    get_openai_client,
    chat,
    get_messages
)

In [ ]:
client = get_openai_client()

In [ ]:
youtube = get_authenticated_youtube_api()
last_n = 20
last_n_videos = get_last_n_videos_with_views(youtube, last_n)
VIDEO_ID = 'SET_YOUTUBE_VIDEO_ID_HERE'

In [ ]:
from flask import Flask, render_template_string, request
import json

app = Flask(__name__)

# HTML template
HTML = """
<!DOCTYPE html>
<html>
    <head>
        <title>Flask Text Processing</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                font-size: 18px;
                line-height: 1.6;
            }
            h1 {
                font-size: 36px;
            }
            h2 {
                font-size: 28px;
            }
            label {
                font-size: 20px;
            }
            input {
                font-size: 18px;
                padding: 5px;
                width: 600px;
            }
            button {
                font-size: 18px;
                padding: 5px 10px;
            }
            p {
                font-size: 20px;
            }
        </style>
    </head>
    <body>
        <h1>Text Processor</h1>
        <form method="post">
            <label for="title">New Video Description:</label>
            <input type="text" id="title" name="desc" value="{{ desc or '' }}" required>
            <button type="submit">Submit</button>
        </form>
        {% if new_title and reasoning and url%}
            <h2>New Title:</h2>
            <p>{{ new_title }}</p>
            <h2>Reasoning:</h2>
            <p>{{ reasoning }}</p>
            <h2>Video URL:</h2>
            <p>{{ url }}</p>
        {% endif %}
    </body>
</html>
"""

In [ ]:
@app.route("/", methods=["GET", "POST"])
def home():
    desc = "i am making a video on martingales"
    output = None
    new_title = None
    reasoning = None
    url = None
    if request.method == "POST":
        # Get the input text from the form
        desc = request.form.get("desc")
        # Call your function f with the input text
        messages = get_messages(last_n_videos, desc)
        output = chat(client, messages)
        output = json.loads(output)
        new_title = output['new_title']
        reasoning = output['reasoning']
        update_video_title(youtube, VIDEO_ID, new_title)
        url = f'https://www.youtube.com/watch?v={VIDEO_ID}'
    # Render the page with the current_title and output
    return render_template_string(HTML, desc=desc, new_title=new_title, reasoning=reasoning, url=url)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5001)